In [2]:
import praw
from psaw import PushshiftAPI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

In [7]:
# reddit = praw.Reddit(
#     client_id="vpl8fF7yAO9qrdkNVlA2dA",
#     client_secret="EPyMGbpzo4qnF5OIYmBAlDKZUX52Vw",
#     username='reddinaat',
#     password='pBUwm2wWN8j5$GXqhc7SmC568',
#     user_agent="webscrape testing",
# )

In [14]:
# subreddit = reddit.subreddit('climatechange')

In [19]:
# hot = subreddit.top(limit=None)
# # x = next(hot)
   
    
# headlines = set()
# for submission in hot:
#     headlines.add(submission.title)

# print(len(headlines))

993


In [3]:
import pandas as pd
from pmaw import PushshiftAPI
api = PushshiftAPI()

#### testing timestamps

In [4]:
import datetime as dt
before = int(dt.datetime(2016,12,31,0,0).timestamp())
after = int(dt.datetime(2015,12,31,0,0).timestamp())

## Select sub reddit

In [5]:
subreddit="climate"
limit=1000
comments = api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
print(f'Retrieved {len(comments)} comments from Pushshift')

Retrieved 1000 comments from Pushshift


#### Show data example

In [6]:
comments_df = pd.DataFrame(comments)
# preview the comments data
comments_df.head(5)

,id,author,controversiality,score,author_flair_text,distinguished,subreddit_id,parent_id,author_flair_css_class,gilded,...,stickied,body,created_utc,subreddit,retrieved_utc,updated_utc,body_sha1,nest_level,utc_datetime_str,edited_on
0,dbt9b5l,Hex0811,0,4,None,None,t5_2qhx3,NaN,None,0,...,False,Is this the same area that Dicaprio goes to in...,1483138536,climate,1484161745,1668780178,1de42784a8942b2bf527bf6bce316f5687cfc616,1.0,2016-12-30 22:55:36,NaN
1,dbt8bn1,experts_never_lie,0,7,None,None,t5_2qhx3,2.901222e+10,None,0,...,False,That's the way science is supposed to work: su...,1483137132,climate,1484161274,1668780177,1a2fa680291c091664a17b85eec4cb489e933c7b,NaN,2016-12-30 22:32:12,NaN
2,dbt77zs,greengordon,0,6,None,None,t5_2qhx3,2.901179e+10,None,0,...,False,I fail to see why a SCIENTIFIC QUALITY (why al...,1483135575,climate,1484160730,1668780175,cf57b186dc7c67bebbb5ea78b9119b11a9e7b2d5,NaN,2016-12-30 22:06:15,NaN
3,dbt6yuf,MicaMtnGirl,0,1,None,None,t5_2qhx3,2.901216e+10,None,0,...,False,Agreed. It also isn't a solution until the sca...,1483135224,climate,1484160603,1668780175,f1cb0ed7fe3924c4b2eb847b6a71239888fb659e,NaN,2016-12-30 22:00:24,NaN
4,dbt4nyy,spiffybaldguy,0,9,None,None,t5_2qhx3,NaN,None,0,...,False,This is a great story. I find it interesting t...,1483132105,climate,1484159487,1668780174,abef3d17988b7d84ca48b247b01d65f4d7c8d3f4,1.0,2016-12-30 21:08:25,NaN


## Start scraping
Scraping the above selected subreddit for every day. Then saving it in a csv for per year.

In [7]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
                      
# cols = ['author_created_utc', 'author', 'body', 'created_utc', 'edited', 'id', 'link_id', 'permalink', 'score', 'subreddit', 'subreddit_id']
dfTesting = pd.DataFrame([])

temp_df = pd.DataFrame() #Temporary empty dataframe

for m in range(1, 13):
    year = 2022
    yearAfter = year
    monthBefore = m
    monthAfter = m
        
    day = 32
    
    
    if (m == 2): #februari 
        day = 29
    if (m == 4) or (m == 6) or (m == 9) or (m == 11): # months with 31 days
        day = 31
        
    for d in range(1, day):
        # NEW

        # date variables
        dayBefore = d
        dayAfter = d-1

        # change dayAfter / monthAfter on certain conditions
        if(dayAfter == 0):
            monthAfter = m-1
            if(m == 2):
                dayAfter = 31
            elif(m==3):
                dayAfter = 28
            elif (m == 4) or (m == 6) or (m == 9) or (m == 11):
                dayAfter = 31
            else: dayAfter = 30
        else: monthAfter = m
            
        if(monthAfter == 0):
            monthAfter = 12
            yearAfter = year-1
        else: yearAfter = year

        #set time frame for scraping -> Scraping for every day in the year.
        print('datebefore', dayBefore, '-', monthBefore, '- ', year, 'date after', dayAfter, '-', monthAfter, '- ', yearAfter)
        before = int(dt.datetime(year,monthBefore,dayBefore,0,0).timestamp())
        after = int(dt.datetime(yearAfter,monthAfter,dayAfter,0,0).timestamp())
        

        # Scrape all comments per day
        comments = api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
        print(f'Retrieved {len(comments)} comments from Pushshift')

        # convert comments into a datafrom
        comments_df = pd.DataFrame(comments)

        #select only the useful columns 
#         datapoints = comments_df[['author_created_utc', 'author', 'body', 'created_utc', 'edited', 'id', 'link_id', 'permalink', 'score', 'subreddit', 'subreddit_id', ]]

        #add all datapoints to a temporary dataframe
        temp_df = temp_df.append(comments_df, ignore_index=True)
    
# all datapoints in temporary dataframe to one dataframe
df1 = pd.DataFrame(temp_df)
df1.head(5)

# Create a csv file
df1.to_csv('reddit_data.csv', encoding='utf-8', header=True, index=False)


datebefore 1 - 1 -  2022 date after 30 - 12 -  2021
Retrieved 539 comments from Pushshift
datebefore 2 - 1 -  2022 date after 1 - 1 -  2022
Retrieved 224 comments from Pushshift
datebefore 3 - 1 -  2022 date after 2 - 1 -  2022
Retrieved 131 comments from Pushshift
datebefore 4 - 1 -  2022 date after 3 - 1 -  2022
Retrieved 317 comments from Pushshift
datebefore 5 - 1 -  2022 date after 4 - 1 -  2022
Retrieved 270 comments from Pushshift
datebefore 6 - 1 -  2022 date after 5 - 1 -  2022
Retrieved 128 comments from Pushshift
datebefore 7 - 1 -  2022 date after 6 - 1 -  2022
Retrieved 273 comments from Pushshift
datebefore 8 - 1 -  2022 date after 7 - 1 -  2022
Retrieved 245 comments from Pushshift
datebefore 9 - 1 -  2022 date after 8 - 1 -  2022
Retrieved 269 comments from Pushshift
datebefore 10 - 1 -  2022 date after 9 - 1 -  2022
Retrieved 229 comments from Pushshift
datebefore 11 - 1 -  2022 date after 10 - 1 -  2022
Retrieved 198 comments from Pushshift
datebefore 12 - 1 -  2022 d

Not all PushShift shards are active. Query results may be incomplete.


Retrieved 0 comments from Pushshift
datebefore 2 - 3 -  2022 date after 1 - 3 -  2022
Retrieved 282 comments from Pushshift
datebefore 3 - 3 -  2022 date after 2 - 3 -  2022
Retrieved 105 comments from Pushshift
datebefore 4 - 3 -  2022 date after 3 - 3 -  2022
Retrieved 70 comments from Pushshift
datebefore 5 - 3 -  2022 date after 4 - 3 -  2022
Retrieved 107 comments from Pushshift
datebefore 6 - 3 -  2022 date after 5 - 3 -  2022
Retrieved 55 comments from Pushshift
datebefore 7 - 3 -  2022 date after 6 - 3 -  2022
Retrieved 112 comments from Pushshift
datebefore 8 - 3 -  2022 date after 7 - 3 -  2022
Retrieved 131 comments from Pushshift
datebefore 9 - 3 -  2022 date after 8 - 3 -  2022
Retrieved 97 comments from Pushshift
datebefore 10 - 3 -  2022 date after 9 - 3 -  2022
Retrieved 146 comments from Pushshift
datebefore 11 - 3 -  2022 date after 10 - 3 -  2022
Retrieved 98 comments from Pushshift
datebefore 12 - 3 -  2022 date after 11 - 3 -  2022
Retrieved 106 comments from Pushs

Not all PushShift shards are active. Query results may be incomplete.


Retrieved 0 comments from Pushshift
datebefore 19 - 3 -  2022 date after 18 - 3 -  2022
Retrieved 137 comments from Pushshift
datebefore 20 - 3 -  2022 date after 19 - 3 -  2022
Retrieved 193 comments from Pushshift
datebefore 21 - 3 -  2022 date after 20 - 3 -  2022
Retrieved 193 comments from Pushshift
datebefore 22 - 3 -  2022 date after 21 - 3 -  2022
Retrieved 124 comments from Pushshift
datebefore 23 - 3 -  2022 date after 22 - 3 -  2022
Retrieved 193 comments from Pushshift
datebefore 24 - 3 -  2022 date after 23 - 3 -  2022
Retrieved 205 comments from Pushshift
datebefore 25 - 3 -  2022 date after 24 - 3 -  2022
Retrieved 132 comments from Pushshift
datebefore 26 - 3 -  2022 date after 25 - 3 -  2022
Retrieved 111 comments from Pushshift
datebefore 27 - 3 -  2022 date after 26 - 3 -  2022
Retrieved 96 comments from Pushshift
datebefore 28 - 3 -  2022 date after 27 - 3 -  2022
Retrieved 166 comments from Pushshift
datebefore 29 - 3 -  2022 date after 28 - 3 -  2022
Retrieved 335

Retrieved 500 comments from Pushshift
datebefore 19 - 6 -  2022 date after 18 - 6 -  2022
Retrieved 111 comments from Pushshift
datebefore 20 - 6 -  2022 date after 19 - 6 -  2022
Retrieved 199 comments from Pushshift
datebefore 21 - 6 -  2022 date after 20 - 6 -  2022
Retrieved 122 comments from Pushshift
datebefore 22 - 6 -  2022 date after 21 - 6 -  2022
Retrieved 531 comments from Pushshift
datebefore 23 - 6 -  2022 date after 22 - 6 -  2022
Retrieved 347 comments from Pushshift
datebefore 24 - 6 -  2022 date after 23 - 6 -  2022
Retrieved 163 comments from Pushshift
datebefore 25 - 6 -  2022 date after 24 - 6 -  2022
Retrieved 173 comments from Pushshift
datebefore 26 - 6 -  2022 date after 25 - 6 -  2022
Retrieved 130 comments from Pushshift
datebefore 27 - 6 -  2022 date after 26 - 6 -  2022
Retrieved 116 comments from Pushshift
datebefore 28 - 6 -  2022 date after 27 - 6 -  2022
Retrieved 106 comments from Pushshift
datebefore 29 - 6 -  2022 date after 28 - 6 -  2022
Retrieved 

Retrieved 270 comments from Pushshift
datebefore 19 - 9 -  2022 date after 18 - 9 -  2022
Retrieved 156 comments from Pushshift
datebefore 20 - 9 -  2022 date after 19 - 9 -  2022
Retrieved 248 comments from Pushshift
datebefore 21 - 9 -  2022 date after 20 - 9 -  2022
Retrieved 325 comments from Pushshift
datebefore 22 - 9 -  2022 date after 21 - 9 -  2022
Retrieved 227 comments from Pushshift
datebefore 23 - 9 -  2022 date after 22 - 9 -  2022
Retrieved 179 comments from Pushshift
datebefore 24 - 9 -  2022 date after 23 - 9 -  2022
Retrieved 203 comments from Pushshift
datebefore 25 - 9 -  2022 date after 24 - 9 -  2022
Retrieved 95 comments from Pushshift
datebefore 26 - 9 -  2022 date after 25 - 9 -  2022
Retrieved 157 comments from Pushshift
datebefore 27 - 9 -  2022 date after 26 - 9 -  2022
Retrieved 83 comments from Pushshift
datebefore 28 - 9 -  2022 date after 27 - 9 -  2022
Retrieved 208 comments from Pushshift
datebefore 29 - 9 -  2022 date after 28 - 9 -  2022
Retrieved 21

datebefore 18 - 12 -  2022 date after 17 - 12 -  2022
Retrieved 326 comments from Pushshift
datebefore 19 - 12 -  2022 date after 18 - 12 -  2022
Retrieved 211 comments from Pushshift
datebefore 20 - 12 -  2022 date after 19 - 12 -  2022
Retrieved 175 comments from Pushshift
datebefore 21 - 12 -  2022 date after 20 - 12 -  2022
Retrieved 337 comments from Pushshift
datebefore 22 - 12 -  2022 date after 21 - 12 -  2022
Retrieved 306 comments from Pushshift
datebefore 23 - 12 -  2022 date after 22 - 12 -  2022
Retrieved 514 comments from Pushshift
datebefore 24 - 12 -  2022 date after 23 - 12 -  2022
Retrieved 324 comments from Pushshift
datebefore 25 - 12 -  2022 date after 24 - 12 -  2022
Retrieved 224 comments from Pushshift
datebefore 26 - 12 -  2022 date after 25 - 12 -  2022
Retrieved 223 comments from Pushshift
datebefore 27 - 12 -  2022 date after 26 - 12 -  2022
Retrieved 201 comments from Pushshift
datebefore 28 - 12 -  2022 date after 27 - 12 -  2022
Retrieved 182 comments fro

In [8]:
# The amount of comments in the year
print(len(df1))

33579


In [50]:
# comments_df.to_csv('./wsb_comments.csv', header=True, index=False, columns=list(comments_df.axes[1]))

In [53]:
# df2 = comments_df[['author_created_utc', 'author', 'body', 'created_utc', 'edited', 'id', 'link_id', 'permalink', 'score', 'subreddit', 'subreddit_id', ]]
# df2.to_csv('reddit_data.csv', encoding='utf-8', header=True, index=False)

## Testing loops without scraping

In [11]:
for m in range(1, 13):
    year = 2021
    yearAfter = year
    monthBefore = m
    monthAfter = m
        
    day = 32
    
    if (m == 2):
        day = 29
    if (m == 4) or (m == 6) or (m == 9) or (m == 11):
        day = 31
    
    for d in range(1, day):
        # loop through the dates :TODO

        # date variables
        dayBefore = d
        dayAfter = d-1

        # if current day is 1, then the previous day is the last day in previous month
        if(dayAfter == 0):
            monthAfter = m-1
            if(m == 2):
                dayAfter = 31
            elif(m==3):
                dayAfter = 28
            elif (m == 4) or (m == 6) or (m == 9) or (m == 11):
                dayAfter = 31
            else: dayAfter = 30
        else: monthAfter = m
            

        if(monthAfter == 0):
            print('hjello')
            monthAfter = 12
            yearAfter = year-1
        else: yearAfter = year
            
        
        print('date before', dayBefore, '-', monthBefore, '- ', year)
        print('date after', dayAfter, '-', monthAfter, '- ', yearAfter)

hjello
date before 1 - 1 -  2021
date after 30 - 12 -  2020
date before 2 - 1 -  2021
date after 1 - 1 -  2021
date before 3 - 1 -  2021
date after 2 - 1 -  2021
date before 4 - 1 -  2021
date after 3 - 1 -  2021
date before 5 - 1 -  2021
date after 4 - 1 -  2021
date before 6 - 1 -  2021
date after 5 - 1 -  2021
date before 7 - 1 -  2021
date after 6 - 1 -  2021
date before 8 - 1 -  2021
date after 7 - 1 -  2021
date before 9 - 1 -  2021
date after 8 - 1 -  2021
date before 10 - 1 -  2021
date after 9 - 1 -  2021
date before 11 - 1 -  2021
date after 10 - 1 -  2021
date before 12 - 1 -  2021
date after 11 - 1 -  2021
date before 13 - 1 -  2021
date after 12 - 1 -  2021
date before 14 - 1 -  2021
date after 13 - 1 -  2021
date before 15 - 1 -  2021
date after 14 - 1 -  2021
date before 16 - 1 -  2021
date after 15 - 1 -  2021
date before 17 - 1 -  2021
date after 16 - 1 -  2021
date before 18 - 1 -  2021
date after 17 - 1 -  2021
date before 19 - 1 -  2021
date after 18 - 1 -  2021
dat